In [ ]:
# Load your dataset
with open("quadrigrams.txt", "r") as f:
    text = f.read().lower().strip().split()

tokens = text
print("Total tokens:", len(tokens))
print("Sample:", tokens[:30])


Total tokens: 13020645
Sample: ['का', 'कहना', 'है', 'कि', '848', 'किया', 'जा', 'रहा', 'है', '557', 'की', 'जा', 'रही', 'है', '469', 'है', 'उन्होंने', 'कहा', 'कि', '428', 'जा', 'रहा', 'है', 'कि', '373', 'ने', 'कहा', 'है', 'कि', '347']


In [ ]:
from nltk.util import ngrams
from collections import Counter

def build_ngram_counts(tokens, n=4):
    counts = {i: Counter(ngrams(tokens, i)) for i in range(1, n+1)}
    return counts

counts = build_ngram_counts(tokens, 4)
print("Sample 4-grams:", list(counts[4].items())[:10])


Sample 4-grams: [(('का', 'कहना', 'है', 'कि'), 1), (('कहना', 'है', 'कि', '848'), 1), (('है', 'कि', '848', 'किया'), 1), (('कि', '848', 'किया', 'जा'), 1), (('848', 'किया', 'जा', 'रहा'), 1), (('किया', 'जा', 'रहा', 'है'), 1), (('जा', 'रहा', 'है', '557'), 1), (('रहा', 'है', '557', 'की'), 1), (('है', '557', 'की', 'जा'), 1), (('557', 'की', 'जा', 'रही'), 1)]


In [ ]:
from collections import defaultdict

def compute_ngram_probs(counts_higher, counts_lower, discount=0.5):
    probs = {}
    for ngram, c in counts_higher.items():
        prefix = ngram[:-1]
        c_prefix = counts_lower.get(prefix, 0)
        if c_prefix > 0:
            probs[ngram] = max(c - discount, 0) / c_prefix
    return probs

def compute_backoff_weights(counts_higher, counts_lower, probs_higher, probs_lower):
    backoff = defaultdict(float)
    contexts = set([ngram[:-1] for ngram in counts_higher.keys()])
    for h in contexts:
        seen_words = [ngram[-1] for ngram in counts_higher.keys() if ngram[:-1]==h]
        sum_disc = sum(probs_higher[(h+(w,))] for w in seen_words if (h+(w,)) in probs_higher)
        sum_lower = sum(probs_lower.get((h[1:],w), 0) for w in seen_words)  # backoff context
        if (1 - sum_lower) > 0:
            backoff[h] = (1 - sum_disc) / (1 - sum_lower)
        else:
            backoff[h] = 1.0
    return backoff

def katz_prob(word, context, probs4, probs3, probs2, probs1, back4, back3, back2):
    # context = tuple of previous words (up to 3)
    if len(context) >= 3 and (context + (word,)) in probs4:
        return probs4[context + (word,)]
    elif len(context) >= 3 and context in back4:
        return back4[context] * katz_prob(word, context[1:], probs3, probs2, probs1, back3, back2, {})
    elif len(context) >= 2 and (context + (word,)) in probs3:
        return probs3[context + (word,)]
    elif len(context) >= 2 and context in back3:
        return back3[context] * katz_prob(word, context[1:], probs2, probs1, {}, {}, {}, {})
    elif len(context) >= 1 and (context + (word,)) in probs2:
        return probs2[context + (word,)]
    elif len(context) >= 1 and context in back2:
        return back2[context] * probs1.get((word,), 1e-8)
    else:
        return probs1.get((word,), 1e-8)


In [ ]:
#!/usr/bin/env python3
"""
Task: Load up to 10,000 sentences from 'sentence_tokens.txt'
and split them into Train / Validation / Test sets.

 - Validation Set: 1,000 sentences
 - Test Set: 1,000 sentences
 - Training Set: remaining 8,000 sentences

Outputs:
    - train.pkl (list of token lists)
    - val.pkl
    - test.pkl
    - split_summary.txt
"""

import os, pickle, random

INPUT = "sentence_tokens.txt"
VAL_SIZE = 1000
TEST_SIZE = 1000
MAX_SENTENCES = 10000
SEED = 42

def load_sentences_txt(path):
    """Load sentences from .txt and tokenize by whitespace."""
    with open(path, "r", encoding="utf8") as fh:
        sentences = [line.strip().split() for line in fh if line.strip()]
    # take only first 10,000
    return sentences[:MAX_SENTENCES]

def save_pickle(obj, fn):
    with open(fn, "wb") as fh:
        pickle.dump(obj, fh)

def main():
    if not os.path.exists(INPUT):
        raise FileNotFoundError(f"Could not find {INPUT}")

    sentences = load_sentences_txt(INPUT)
    total = len(sentences)
    print(f"Loaded {total} sentences from {INPUT}")

    if total < VAL_SIZE + TEST_SIZE:
        raise ValueError("Not enough sentences to create the requested splits.")

    random.seed(SEED)
    random.shuffle(sentences)

    val = sentences[:VAL_SIZE]
    test = sentences[VAL_SIZE:VAL_SIZE + TEST_SIZE]
    train = sentences[VAL_SIZE + TEST_SIZE:]

    save_pickle(train, "train.pkl")
    save_pickle(val, "val.pkl")
    save_pickle(test, "test.pkl")

    print(f"✅ train.pkl → {len(train)} sentences")
    print(f"✅ val.pkl   → {len(val)} sentences")
    print(f"✅ test.pkl  → {len(test)} sentences")

    # Write a summary text file
    with open("split_summary.txt", "w", encoding="utf8") as fh:
        fh.write(f"Total sentences (used): {total}\n")
        fh.write(f"Train: {len(train)}\nVal: {len(val)}\nTest: {len(test)}\n")

    print("📄 Wrote split_summary.txt")

if __name__ == "__main__":
    main()


Loaded 10000 sentences from sentence_tokens.txt
✅ train.pkl → 8000 sentences
✅ val.pkl   → 1000 sentences
✅ test.pkl  → 1000 sentences
📄 Wrote split_summary.txt


In [ ]:
#!/usr/bin/env python3
"""
Generate unigram/bigram/trigram/quadrigram count files from train.pkl
Outputs: counts_n1.pkl ... counts_n4.pkl
"""

import pickle
from collections import Counter
from pathlib import Path

TRAIN_PICKLE = "train.pkl"
MAX_N = 4
SPECIAL_START = "<s>"
SPECIAL_END = "</s>"

def load_pickle(fn):
    with open(fn, "rb") as fh:
        return pickle.load(fh)

def ngrams_from_sentence(tokens, n):
    padded = [SPECIAL_START] * (n - 1) + tokens + [SPECIAL_END]
    return [tuple(padded[i:i + n]) for i in range(len(padded) - n + 1)]

def build_counts(sentences, n):
    c = Counter()
    for s in sentences:
        ngs = ngrams_from_sentence(s, n)
        c.update(ngs)
    return c

def save_pickle(obj, fn):
    with open(fn, "wb") as fh:
        pickle.dump(obj, fh)

def main():
    train = load_pickle(TRAIN_PICKLE)
    print(f"Loaded train ({len(train)} sentences). Building n-gram counts...")
    for n in range(1, MAX_N + 1):
        counts = build_counts(train, n)
        save_pickle(counts, f"counts_n{n}.pkl")
        print(f"Saved counts_n{n}.pkl with {len(counts)} unique n-grams")
    print("Done. Generated counts_n1.pkl to counts_n4.pkl")

if __name__ == "__main__":
    main()


Loaded train (8000 sentences). Building n-gram counts...
Saved counts_n1.pkl with 18172 unique n-grams
Saved counts_n2.pkl with 94259 unique n-grams
Saved counts_n3.pkl with 143362 unique n-grams
Saved counts_n4.pkl with 159153 unique n-grams
Done. Generated counts_n1.pkl to counts_n4.pkl


In [ ]:
counts_n1=[]
counts_n3=[]
counts_n4=[]
counts_n2=[]
with open("counts_n1.pkl", "rb") as f:
    counts_n1 = pickle.load(f)  # for unigram or bigram as per your data
with open("counts_n2.pkl", "rb") as f:
    counts_n2 = pickle.load(f)
with open("counts_n3.pkl", "rb") as f:
    counts_n3 = pickle.load(f)
with open("counts_n4.pkl", "rb") as f:
    counts_n4 = pickle.load(f)



probs4 = compute_ngram_probs(counts_n4, counts_n3)
probs3 = compute_ngram_probs(counts_n3, counts_n2)
probs2 = compute_ngram_probs(counts_n2, counts_n1)
probs1 = {k: v / sum(counts_n1.values()) for k, v in counts_n1.items()}




print("✅ Computed discounted probabilities and backoff weights successfully.")


✅ Computed discounted probabilities and backoff weights successfully.


In [ ]:
back4 = compute_backoff_weights(counts_n4, counts_n3, probs4, probs3)

In [ ]:
back3 = compute_backoff_weights(counts_n3, counts_n2, probs3, probs2)
back2 = compute_backoff_weights(counts_n2, counts_n1, probs2, probs1)

In [ ]:
print(len(probs1))
print(len(probs2))
print(len(probs3))
print(len(probs4))
print(len(back4))
print(len(back3))
print(len(back2))

18172
91417
140520
156311
139548
93090
18172


In [ ]:
import random
import numpy as np

def generate_sentence_greedy(katz_prob_fn, start_tokens, max_len=20, vocab=None, top_k=10, min_len=5):
    sentence = list(start_tokens)

    for _ in range(max_len):
        context = tuple(sentence[-(len(start_tokens)):])
        # Get probabilities for all words
        word_probs = np.array([katz_prob_fn(w, context) for w in vocab])

        # Normalize to sum=1 (important)
        if word_probs.sum() == 0:
            word_probs = np.ones_like(word_probs) / len(word_probs)
        else:
            word_probs /= word_probs.sum()

        # Get indices of top_k most probable words
        top_indices = np.argsort(word_probs)[-top_k:]
        top_words = [vocab[i] for i in top_indices]
        top_probs = word_probs[top_indices]

        # Re-normalize top-k probabilities
        top_probs = top_probs / top_probs.sum()

        # Randomly choose next word among top-k
        next_word = np.random.choice(top_words, p=top_probs)

        # Avoid early </s>
        if len(sentence) < min_len and next_word == '</s>':
            continue

        sentence.append(next_word)

        if next_word == '</s>':
            break

    return sentence


import numpy as np
import heapq
import random

def generate_sentence_beam(katz_prob_fn, start_tokens, beam_size=20, max_len=20, vocab=None, top_k=10, temperature=1.0):
    beam = [(0.0, list(start_tokens))]  # (neg log-prob, sequence)

    for _ in range(max_len):
        candidates = []
        for logp, seq in beam:
            context = tuple(seq[-(len(start_tokens)):])

            # Compute probabilities for all words
            probs = np.array([katz_prob_fn(w, context) for w in vocab])
            probs = np.maximum(probs, 1e-12)  # Avoid log(0)
            probs = probs / probs.sum()

            # Pick top-k candidates
            top_indices = np.argsort(probs)[-top_k:]
            top_probs = probs[top_indices]
            top_probs = top_probs / top_probs.sum()

            # Randomly sample one next word (weighted by probability)
            sampled_idx = np.random.choice(top_indices, p=top_probs)
            w = vocab[sampled_idx]
            p = probs[sampled_idx]

            candidates.append((logp - np.log(p), seq + [w]))

        # Keep best `beam_size` beams
        beam = heapq.nsmallest(beam_size, candidates, key=lambda x: x[0])

        # Stop if all beams end with </s>
        if all(seq[-1] == '</s>' for _, seq in beam):
            break

    # Randomly pick one of the final top beams (for more diversity)
    return random.choice(beam)[1]


def generate_sentences(katz_prob_fn, vocab, n_sentences=100, start_tokens=None, method="greedy", beam_size=20):
    sentences = []
    if start_tokens is None:
        start_tokens = ['<s>'] * (len(vocab[0].split()) if hasattr(vocab[0], "__iter__") else 3)

    for _ in range(n_sentences):
        if method == "greedy":
            sent = generate_sentence_greedy(katz_prob_fn, start_tokens, vocab=vocab)
        elif method == "beam":
            sent = generate_sentence_beam(katz_prob_fn, start_tokens, beam_size=beam_size, vocab=vocab)
        else:
            raise ValueError("Method must be 'greedy' or 'beam'")
        sentences.append(" ".join(sent))
    return sentences


import numpy as np
import random
import heapq

# Example: Katz probability function for quadrigrams (or any n-gram)
def get_katz_prob_fn(n, probs_list, backoff_list):
    """
    Returns a function f(word, context) → probability
    probs_list = [probs1, probs2, ..., probsN]
    backoff_list = [back1, back2, ..., backN-1]
    """
    def katz_prob(word, context):
        if n == 1:
            return probs_list[0].get((word,), 1e-8)
        else:
            # recursively backoff
            context = context[-(n-1):] if len(context) >= n-1 else context
            if tuple(context + (word,)) in probs_list[n-1]:
                return probs_list[n-1][tuple(context + (word,))]
            elif tuple(context) in backoff_list[n-2]:
                lower_fn = get_katz_prob_fn(n-1, probs_list, backoff_list)
                return backoff_list[n-2][tuple(context)] * lower_fn(word, context[1:])
            else:
                lower_fn = get_katz_prob_fn(n-1, probs_list, backoff_list)
                return lower_fn(word, context[1:])
    return katz_prob

In [ ]:
# probs_list = [probs1, probs2, probs3, probs4]
# backoff_list = [back2, back3, back4]  # backoff weights for 2-,3-,4-grams
vocab = [w[0] for w in counts_n1.keys()]  # unigram vocabulary
start_tokens = ['<s>', '<s>', '<s>']

# Get Katz probability function
katz_fn = get_katz_prob_fn(4, [probs1, probs2, probs3, probs4], [back2,back3, back4])

# Generate 100 greedy sentences
greedy_sentences = generate_sentences(katz_fn, vocab, n_sentences=10, start_tokens=start_tokens, method="greedy")

# Optionally save to files
with open("quadrigram_greedy.txt", "w", encoding="utf-8") as f:
    for s in greedy_sentences:
        f.write(s + "\n")




In [ ]:
# Generate 100 beam search sentences
beam_sentences = generate_sentences(katz_fn, vocab, n_sentences=10, start_tokens=start_tokens, method="beam", beam_size=20)

with open("quadrigram_beam.txt", "w", encoding="utf-8") as f:
    for s in beam_sentences:
        f.write(s + "\n")

In [ ]:
import random
import numpy as np

def generate_sentence_greedy(katz_prob_fn, start_tokens, max_len=20, vocab=None, top_k=10, min_len=5):
    sentence = list(start_tokens)

    for _ in range(max_len):
        context = tuple(sentence[-(len(start_tokens)):])
        # Get probabilities for all words
        word_probs = np.array([katz_prob_fn(w, context) for w in vocab])

        # Normalize to sum=1 (important)
        if word_probs.sum() == 0:
            word_probs = np.ones_like(word_probs) / len(word_probs)
        else:
            word_probs /= word_probs.sum()

        # Get indices of top_k most probable words
        top_indices = np.argsort(word_probs)[-top_k:]
        top_words = [vocab[i] for i in top_indices]
        top_probs = word_probs[top_indices]

        # Re-normalize top-k probabilities
        top_probs = top_probs / top_probs.sum()

        # Randomly choose next word among top-k
        next_word = np.random.choice(top_words, p=top_probs)

        # Avoid early </s>
        if len(sentence) < min_len and next_word == '</s>':
            continue

        sentence.append(next_word)

        if next_word == '</s>':
            break

    return sentence


import numpy as np
import heapq
import random

def generate_sentence_beam(katz_prob_fn, start_tokens, beam_size=20, max_len=20, vocab=None, top_k=10, temperature=1.0):
    beam = [(0.0, list(start_tokens))]  # (neg log-prob, sequence)

    for _ in range(max_len):
        candidates = []
        for logp, seq in beam:
            context = tuple(seq[-(len(start_tokens)):])

            # Compute probabilities for all words
            probs = np.array([katz_prob_fn(w, context) for w in vocab])
            probs = np.maximum(probs, 1e-12)  # Avoid log(0)
            probs = probs / probs.sum()

            # Pick top-k candidates
            top_indices = np.argsort(probs)[-top_k:]
            top_probs = probs[top_indices]
            top_probs = top_probs / top_probs.sum()

            # Randomly sample one next word (weighted by probability)
            sampled_idx = np.random.choice(top_indices, p=top_probs)
            w = vocab[sampled_idx]
            p = probs[sampled_idx]

            candidates.append((logp - np.log(p), seq + [w]))

        # Keep best `beam_size` beams
        beam = heapq.nsmallest(beam_size, candidates, key=lambda x: x[0])

        # Stop if all beams end with </s>
        if all(seq[-1] == '</s>' for _, seq in beam):
            break

    # Randomly pick one of the final top beams (for more diversity)
    return random.choice(beam)[1]


def generate_sentences(katz_prob_fn, vocab, n_sentences=100, start_tokens=None, method="greedy", beam_size=20):
    sentences = []
    if start_tokens is None:
        start_tokens = ['<s>'] * (len(vocab[0].split()) if hasattr(vocab[0], "__iter__") else 3)

    for _ in range(n_sentences):
        if method == "greedy":
            sent = generate_sentence_greedy(katz_prob_fn, start_tokens, vocab=vocab)
        elif method == "beam":
            sent = generate_sentence_beam(katz_prob_fn, start_tokens, beam_size=beam_size, vocab=vocab)
        else:
            raise ValueError("Method must be 'greedy' or 'beam'")
        sentences.append(" ".join(sent))
    return sentences


import numpy as np
import random
import heapq

# Example: Katz probability function for quadrigrams (or any n-gram)
def get_katz_prob_fn(n, probs_list, backoff_list):
    """
    Returns a function f(word, context) → probability
    probs_list = [probs1, probs2, ..., probsN]
    backoff_list = [back1, back2, ..., backN-1]
    """
    def katz_prob(word, context):
        if n == 1:
            return probs_list[0].get((word,), 1e-8)
        else:
            # recursively backoff
            context = context[-(n-1):] if len(context) >= n-1 else context
            if tuple(context + (word,)) in probs_list[n-1]:
                return probs_list[n-1][tuple(context + (word,))]
            elif tuple(context) in backoff_list[n-2]:
                lower_fn = get_katz_prob_fn(n-1, probs_list, backoff_list)
                return backoff_list[n-2][tuple(context)] * lower_fn(word, context[1:])
            else:
                lower_fn = get_katz_prob_fn(n-1, probs_list, backoff_list)
                return lower_fn(word, context[1:])
    return katz_prob

In [ ]:
import pandas as pd
from collections import Counter

# Suppose you already have a dictionary of n-gram counts:
# e.g., counts = {('the',): 123, ('of',): 234, ('new',): 5, ...}
# Replace with your actual counts dictionary
counts=[]
with open("counts_n1.pkl", "rb") as f:
    counts = pickle.load(f)  # for unigram or bigram as per your data

# Step 1: Count frequencies of frequencies
freq_counter = Counter(counts.values())  # Nc table

# Step 2: Sort by observed count
sorted_counts = sorted(freq_counter.items())

# Step 3: Compute C* using Good-Turing formula
rows = []
for c, Nc in sorted_counts[:100]:  # top 100 frequency levels
    Nc1 = freq_counter.get(c + 1, 0)
    if Nc > 0:
        c_star = (c + 1) * (Nc1 / Nc)
    else:
        c_star = 0
    rows.append((c, Nc, round(c_star, 4)))

# Step 4: Convert to DataFrame for pretty table
gt_table = pd.DataFrame(rows, columns=["C (Observed Count)", "Nc (Frequency of Count)", "C* (Adjusted Count)"])
print(gt_table)

FileNotFoundError: [Errno 2] No such file or directory: 'counts_n1.pkl'